In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

os.system('pip install tensorflow-gpu==1.14.0')
os.system('pip install keras==2.2.5')

import random
import numpy as np
from collections import deque
# from keras.models import Sequential, load_model, Model
# from keras.layers import Dense, Dropout, Lambda, Input, Subtract, Add
# from keras.optimizers import Adam
# from keras.utils import to_categorical
# from keras import losses
# from keras import initializers, regularizers
# import keras.backend as K

from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Dropout, Lambda, Input, Subtract, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import losses
from tensorflow.keras import initializers, regularizers
import tensorflow.keras.backend as K


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import shap
from scipy import stats
import collections
#from treatments import hypertension_treatments, dm_treatment, ascvd_treatment, hypertension_treatments_history, dm_treatment_history
#from ascvd_risk import framingham_ascvd_risk
diagnosis_reward = False
EPISODES = 20000

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

hypertension_treatments = {'antihypertensives,_ace_inhibitors', 'calcium_channel_blocking_agents',
                           'beta-adrenergic_blocking_agents', 'angiotensin_receptor_antag.-thiazide_diuretic_comb',
                           'antihypertensives,_angiotensin_receptor_antagonist', 'thiazide_and_related_diuretics',
                           'potassium_sparing_diuretics_in_combination', 'alpha/beta-adrenergic_blocking_agents',
                           'antihyperlipid-_hmg-coa_ri-calcium_channel_blocker',
                           'ace_inhibitor-thiazide_or_thiazide-like_diuretic',
                           'angioten.receptr_antag-calcium_chanl_blkr-thiazide',
                           'angiotensin_receptor_blockr-calcium_channel_blockr',
                           'miotics_and_other_intraocular_pressure_reducers',
                           'renin_inhibitor,direct-angiotensin_receptr_antagon',
                           'angiotensin_recept-neprilysin_inhibitor_comb(arni)',
                           'beta-blockers_and_thiazide,thiazide-like_diuretics', 'bulk_chemicals',
                           'renin_inhibitor,direct_and_thiazide_diuretic_comb',
                           'anti-inflammatory,_interleukin-1_beta_blockers'}

dm_treatment = {'antihyperglycemic,_biguanide_type', 'insulins',
                'antihyperglycemic,insulin-release_stim.-biguanide',
                'antihyperglycemic,_dpp-4_inhibitors',
                'antihypergly,incretin_mimetic(glp-1_recep.agonist)',
                'antihyperglycemic,_insulin-release_stimulant_type',
                'antihyperglycemic,dpp-4_inhibitor-biguanide_combs.',
                'antihyperglycemc-sod/gluc_cotransport2(sglt2)inhib',
                'antihyperglycemic,_alpha-glucosidase_inhibitors',
                'antihyperglycemic,_thiazolidinedione_and_biguanide',
                'antihyperglycemic,thiazolidinedione(pparg_agonist)',
                'antihyperglycemic_-_dopamine_receptor_agonists',
                'antihyperglycemic-sglt2_inhibitor-biguanide_combs.',
                'antihyperglycemic,_sglt-2_and_dpp-4_inhibitor_comb',
                'antihyperglycemic,_thiazolidinedione-sulfonylurea',
                'antihypergly,dpp-4_enzyme_inhib.-thiazolidinedione',
                'antihypergly,insulin,long_act-glp-1_recept.agonist',
                'antihyperglycemic,_amylin_analog-type'}

ascvd_treatment = {'antihyperlip.hmg_coa_reduct_inhib-cholest.ab.inhib',
                   'antihyperlipidemic-hmg_coa_reductase_inhib.-niacin',
                   'antihyperlipidemic_-_hmg_coa_reductase_inhibitors',
                   'antihyperlipidemic_-_pcsk9_inhibitors',
                   'bile_salt_sequestrants',
                   'lipotropics'}

hypertension_treatments_history = ['antihypertensives,_ace_inhibitors_cur',
 'calcium_channel_blocking_agents_cur',
 'beta-adrenergic_blocking_agents_cur',
 'angiotensin_receptor_antag.-thiazide_diuretic_comb_cur',
 'antihypertensives,_angiotensin_receptor_antagonist_cur',
 'thiazide_and_related_diuretics_cur',
 'potassium_sparing_diuretics_in_combination_cur',
 'alpha/beta-adrenergic_blocking_agents_cur',
 'antihyperlipid-_hmg-coa_ri-calcium_channel_blocker_cur',
 'ace_inhibitor-thiazide_or_thiazide-like_diuretic_cur',
 'angioten.receptr_antag-calcium_chanl_blkr-thiazide_cur',
 'angiotensin_receptor_blockr-calcium_channel_blockr_cur',
 'miotics_and_other_intraocular_pressure_reducers_cur',
 'renin_inhibitor,direct-angiotensin_receptr_antagon_cur',
 'angiotensin_recept-neprilysin_inhibitor_comb(arni)_cur',
 'beta-blockers_and_thiazide,thiazide-like_diuretics_cur',
 'bulk_chemicals_cur',
 'renin_inhibitor,direct_and_thiazide_diuretic_comb_cur',
 'anti-inflammatory,_interleukin-1_beta_blockers_cur',
 'antihypertensives,_ace_inhibitors_hist',
 'calcium_channel_blocking_agents_hist',
 'beta-adrenergic_blocking_agents_hist',
 'angiotensin_receptor_antag.-thiazide_diuretic_comb_hist',
 'antihypertensives,_angiotensin_receptor_antagonist_hist',
 'thiazide_and_related_diuretics_hist',
 'potassium_sparing_diuretics_in_combination_hist',
 'alpha/beta-adrenergic_blocking_agents_hist',
 'antihyperlipid-_hmg-coa_ri-calcium_channel_blocker_hist',
 'ace_inhibitor-thiazide_or_thiazide-like_diuretic_hist',
 'angioten.receptr_antag-calcium_chanl_blkr-thiazide_hist',
 'angiotensin_receptor_blockr-calcium_channel_blockr_hist',
 'miotics_and_other_intraocular_pressure_reducers_hist',
 'renin_inhibitor,direct-angiotensin_receptr_antagon_hist',
 'angiotensin_recept-neprilysin_inhibitor_comb(arni)_hist',
 'beta-blockers_and_thiazide,thiazide-like_diuretics_hist',
 'bulk_chemicals_hist',
 'renin_inhibitor,direct_and_thiazide_diuretic_comb_hist',
 'anti-inflammatory,_interleukin-1_beta_blockers_hist']

dm_treatment_history = ['antihyperglycemic,_biguanide_type_cur',
 'insulins_cur',
 'antihyperglycemic,insulin-release_stim.-biguanide_cur',
 'antihyperglycemic,_dpp-4_inhibitors_cur',
 'antihypergly,incretin_mimetic(glp-1_recep.agonist)_cur',
 'antihyperglycemic,_insulin-release_stimulant_type_cur',
 'antihyperglycemic,dpp-4_inhibitor-biguanide_combs._cur',
 'antihyperglycemc-sod/gluc_cotransport2(sglt2)inhib_cur',
 'antihyperglycemic,_alpha-glucosidase_inhibitors_cur',
 'antihyperglycemic,_thiazolidinedione_and_biguanide_cur',
 'antihyperglycemic,thiazolidinedione(pparg_agonist)_cur',
 'antihyperglycemic_-_dopamine_receptor_agonists_cur',
 'antihyperglycemic-sglt2_inhibitor-biguanide_combs._cur',
 'antihyperglycemic,_sglt-2_and_dpp-4_inhibitor_comb_cur',
 'antihyperglycemic,_thiazolidinedione-sulfonylurea_cur',
 'antihypergly,dpp-4_enzyme_inhib.-thiazolidinedione_cur',
 'antihypergly,insulin,long_act-glp-1_recept.agonist_cur',
 'antihyperglycemic,_amylin_analog-type_cur',
 'antihyperglycemic,_biguanide_type_hist',
 'insulins_hist',
 'antihyperglycemic,insulin-release_stim.-biguanide_hist',
 'antihyperglycemic,_dpp-4_inhibitors_hist',
 'antihypergly,incretin_mimetic(glp-1_recep.agonist)_hist',
 'antihyperglycemic,_insulin-release_stimulant_type_hist',
 'antihyperglycemic,dpp-4_inhibitor-biguanide_combs._hist',
 'antihyperglycemc-sod/gluc_cotransport2(sglt2)inhib_hist',
 'antihyperglycemic,_alpha-glucosidase_inhibitors_hist',
 'antihyperglycemic,_thiazolidinedione_and_biguanide_hist',
 'antihyperglycemic,thiazolidinedione(pparg_agonist)_hist',
 'antihyperglycemic_-_dopamine_receptor_agonists_hist',
 'antihyperglycemic-sglt2_inhibitor-biguanide_combs._hist',
 'antihyperglycemic,_sglt-2_and_dpp-4_inhibitor_comb_hist',
 'antihyperglycemic,_thiazolidinedione-sulfonylurea_hist',
 'antihypergly,dpp-4_enzyme_inhib.-thiazolidinedione_hist',
 'antihypergly,insulin,long_act-glp-1_recept.agonist_hist',
 'antihyperglycemic,_amylin_analog-type_hist']

def framingham_ascvd_risk(x):
    if x[0] == 0:
        AgeFactor = 2.32888
        TotalCholFactor = 1.20904
        HDLCholFactor = -0.70833
        if x[6]:
            SysBPFactor = 2.82263
        else:
            SysBPFactor = 2.76157

        if x[5] == 1.0:
            Cig = 0.52873
        else:
            Cig = 0.0

        if x[7]:
            DM = 0.69154
        else:
            DM = 0.0
        AvgRisk = 26.1931
        RiskPeriodFactor = 0.95012
    else:
        AgeFactor = 3.06117
        TotalCholFactor = 1.12370
        HDLCholFactor = -0.93263
        if x[6]:
            SysBPFactor = 1.99881
        else:
            SysBPFactor = 1.93303
        if x[5] == 1.0:
            Cig = 0.65451
        else:
            Cig = 0.0

        if x[7]:
            DM = 0.57367
        else:
            DM = 0.0
        AvgRisk = 23.9802
        RiskPeriodFactor = 0.88936

    risk_factors = (np.log(x[1]) * AgeFactor) + (np.log(x[2]) * TotalCholFactor) + (np.log(x[3]) * HDLCholFactor) + (
                np.log(x[4]) * SysBPFactor) + Cig + DM - AvgRisk
    risk = 100 * (1 - np.power(RiskPeriodFactor, np.exp(risk_factors)))
    return (risk)

class DQNAgent:
    def __init__(self, state_size, action_size, targets, state_cols, reward_cols, next_state_cols, class_weights=None, dueling=False):
        self.state_size = state_size
        self.action_size = action_size
        self.targets = targets
        self.state_cols = state_cols
        self.reward_cols = reward_cols
        self.next_state_cols = next_state_cols
        self.dueling = dueling
        self.class_weights = class_weights
        self.hidden_layers = {'layers': [256, 512, 256], 'activation': ['tanh', 'tanh', 'tanh']}
        self.gamma = 0.8  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate_decay = 0.01
        self.learning_rate = 0.0004
        self.l2_reg = 10e-5
        self.model = Sequential()
        self.target_model = Sequential()
        self._build_model()
        self._build_target_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        if self.dueling:
            input_ = Input(shape=( self.state_size,))

            for i in range(len(self.hidden_layers['layers'])):
                if i is 0:
                    x = Dense(self.hidden_layers['layers'][i], input_dim=self.state_size,
                                         activation=self.hidden_layers['activation'][i])(input_)
                    x = Dropout(0.5)(x)
                else:
                    x = Dense(self.hidden_layers['layers'][i], activation=self.hidden_layers['activation'][i])(x)
                    x = Dropout(0.5)(x)

            value = Dense(256, activation="relu")(x)
            value = Dense(1, activation="relu")(value)
            advantage = Dense(256, activation="relu")(x)
            advantage = Dense(self.action_size, activation="relu")(advantage)
            advantage_mean = Lambda(lambda x: K.mean(x, axis=1))(advantage)
            advantage = Subtract()([advantage, advantage_mean])
            out = Add()([value, advantage])

            model = Model(inputs=input_, outputs=out)
            model.compile(optimizer=Adam(lr=self.learning_rate), loss=losses.mse)
            self.model = model
        else:
            for i in range(len(self.hidden_layers['layers'])):
                if i is 0:
                    self.model.add(Dense(self.hidden_layers['layers'][i], input_dim=self.state_size,
                                         activation=self.hidden_layers['activation'][i]))
                    self.model.add(Dropout(0.5))
                else:
                    self.model.add(Dense(self.hidden_layers['layers'][i], activation=self.hidden_layers['activation'][i]))
                    self.model.add(Dropout(0.5))
            self.model.add(Dense(self.action_size, activation='linear'))
            self.model.compile(optimizer=Adam(lr=self.learning_rate), loss=losses.mse)


    def _build_target_model(self):
        if self.dueling:
            input_ = Input(shape=(self.state_size,))

            for i in range(len(self.hidden_layers['layers'])):
                if i is 0:
                    x = Dense(self.hidden_layers['layers'][i], input_dim=self.state_size,
                              activation=self.hidden_layers['activation'][i])(input_)
                    x = Dropout(0.5)(x)
                else:
                    x = Dense(self.hidden_layers['layers'][i], activation=self.hidden_layers['activation'][i])(x)
                    x = Dropout(0.5)(x)

            value = Dense(256, activation="relu")(x)
            value = Dense(1, activation="relu")(value)
            advantage = Dense(256, activation="relu")(x)
            advantage = Dense(self.action_size, activation="relu")(advantage)
            advantage_mean = Lambda(lambda x: K.mean(x, axis=1))(advantage)
            advantage = Subtract()([advantage, advantage_mean])
            out = Add()([value, advantage])

            model = Model(inputs=input_, outputs=out)
            model.compile(optimizer=Adam(lr=self.learning_rate), loss=losses.mse)
            self.target_model = model
        else:
            for i in range(len(self.hidden_layers['layers'])):
                if i is 0:
                    self.target_model.add(Dense(self.hidden_layers['layers'][i], input_dim=self.state_size,
                                                activation=self.hidden_layers['activation'][i]))
                    self.model.add(Dropout(0.5))
                else:
                    self.target_model.add(
                        Dense(self.hidden_layers['layers'][i], activation=self.hidden_layers['activation'][i]))
                    self.model.add(Dropout(0.5))
            self.target_model.add(Dense(self.action_size, activation='linear'))
            self.target_model.compile(optimizer=Adam(lr=self.learning_rate), loss=losses.mse)

    def update_target(self):
        self.target_model.set_weights(self.model.get_weights())

    def _hash_action(self, actions):
        return sum([pow(2, i) for i in range(8) if actions[i] == 1])

    def _to_categorical(self, x):
        to_categorical(x, num_classes=self.action_size)

    def replay(self, minibatch, DDQN):
        states = np.array(minibatch[self.state_cols])
        targets_f = self.model.predict(states)
        for idx in range(minibatch.shape[0]):
            next_state = np.reshape(minibatch[self.next_state_cols].iloc[idx].tolist(), [1, state_size])
            reward = minibatch[self.reward_cols].iloc[idx].values[0]
            action = np.argmax(minibatch[self.targets].iloc[
                                   idx].tolist())  # self._hash_action(minibatch[self.targets].iloc[idx].tolist())
            done = np.isnan(next_state[0, 0])
            if not DDQN:
                # Vanilla DQN
                target = reward + self.gamma * np.max(self.model.predict(next_state)[0]) * np.invert(done)
            else:
                # Double DQN
                action_index = np.argmax(self.model.predict(next_state)[0])
                target = reward + self.gamma * self.target_model.predict(next_state)[0][action_index] * np.invert(done)

            targets_f[idx][action] = target
            # Filtering out states and targets for training

        history = self.model.fit(states, targets_f, epochs=1, verbose=0, class_weight=self.class_weights)
        # Keeping track of loss
        loss = history.history['loss'][0]
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        return loss

    def prioritize(self, state, next_state, action, reward, done, alpha=0.6):
        q_next = reward + self.discount_factor * np.max(self.predict(next_state)[0])
        q = self.predict(state)[0][action]
        p = (np.abs(q_next-q)+ (np.e ** -10)) ** alpha
        self.priority.append(p)
        self.memory.append((state, next_state, action, reward, done))

    def get_priority_experience_batch(self):
        p_sum = np.sum(self.priority)
        prob = self.priority / p_sum
        sample_indices = random.choices(range(len(prob)), k=self.batch_size, weights=prob)
        importance = (1/prob) * (1/len(self.priority))
        importance = np.array(importance)[sample_indices]
        samples = np.array(self.memory)[sample_indices]
        return samples, importance

    def _replay(self):
        """
        experience replay. find the q-value and train the neural network model with state as input and q-values as targets
        :return:
        """
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
        batch, importance = self.get_priority_experience_batch()
        for b, i in zip(batch, importance):
            state, next_state, action, reward, done = b
            target = reward
            if not done:
                target = reward + self.discount_factor * np.max(self.predict(next_state)[0])
            final_target = self.predict(state)
            final_target[0][action] = target
            imp = i ** (1-self.epsilon)
            imp = np.reshape(imp, 1)
            self.fit(state, final_target, imp)

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

    def model_loss(self):
        """" Wrapper function which calculates auxiliary values for the complete loss function.
         Returns a *function* which calculates the complete loss given only the input and target output """
        # KL loss
        kl_loss = self.calculate_kl_loss
        # Reconstruction loss
        md_loss_func = self.calculate_md_loss

        # KL weight (to be used by total loss and by annealing scheduler)
        self.kl_weight = K.variable(self.hps['kl_weight_start'], name='kl_weight')
        kl_weight = self.kl_weight

        def seq2seq_loss(y_true, y_pred):
            """ Final loss calculation function to be passed to optimizer"""
            # Reconstruction loss
            md_loss = md_loss_func(y_true, y_pred)
            # Full loss
            model_loss = kl_weight * kl_loss() + md_loss
            return model_loss

        return seq2seq_loss

def hash_to_action(x):
    return int(''.join(map(str, x)))
    # return sum([int(x[i]) * (2**i) for i in range(len(x))])

def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

def decode_to_treatment(x, labels):
    gap = len(labels) - len(x)
    index_temp = findOccurrences(x, '1')
    output = [labels[idx + gap] for idx in index_temp]
    return output

def map_to_disease_category(x, disease, label_cols):
    for t in range(len(x)):
        if x[t] == 1 and label_cols[t] in treatment_ctg[disease]:
            return True
    return False

all_treamments = hypertension_treatments.union(dm_treatment).union(ascvd_treatment)
cur_next_treatments = set(["next_" + i for i in all_treamments]).union(all_treamments)

treatment_ctg = {'hypertension': hypertension_treatments, 'diabetes': dm_treatment, 'ascvd': ascvd_treatment}

target_treatment = 'ascvd'
action_size = 100

excluded = ['study_id', 'encounter_dt_ran']

data_path = '/kaggle/input/cleaned-ehr-treatment-v6/cleaned_EHR_treatment_param_lab_test_final_3diseases-v6.csv'
# '~/Research/PHD/project/Hua Zheng/previous code/cleaned_EHR_treatment_param_lab_test_final_3diseases_cvd_encounter_diagnosis.csv'
data = pd.read_csv(data_path)
if diagnosis_reward:
    data['reward'] = data['reward_diagnosis']
    data.drop('reward_diagnosis', inplace=True, axis=1)

#%% weight different rewards
reward_weight = [0, 1, 0]
data['reward'] = data.apply(lambda x: (x['reward_bp'] * reward_weight[0] + x['reward_ascvd'] * reward_weight[1] + x['reward_diabetes'] * reward_weight[2]) / sum(reward_weight), axis = 1) # (data[['reward_bp']] * 2 + data[['reward_ascvd']] + data[['reward_ascvd']] * 2) / 5#, 'reward_ascvd', 'reward_diabetes'
#    data = data.drop(['egfr_mdrd_african_american_min', 'egfr_mdrd_african_american_max', 'egfr_mdrd_african_american', 'egfr_mdrd_non_african_american', 'egfr_mdrd_non_african_american_max','egfr_mdrd_non_african_american_min', 'next_egfr_mdrd_african_american','next_egfr_mdrd_african_american_max','next_egfr_mdrd_african_american_min', 'next_egfr_mdrd_non_african_american','next_egfr_mdrd_non_african_american_max', 'next_egfr_mdrd_non_african_american_min', 'bulk_chemicals_hist', 'next_bulk_chemicals_hist'],axis=1)
data = data.drop(['egfr_mdrd_african_american_min', 'egfr_mdrd_african_american_max', 'egfr_mdrd_african_american',
                  'egfr_mdrd_non_african_american', 'egfr_mdrd_non_african_american_max',
                  'egfr_mdrd_non_african_american_min', 'next_egfr_mdrd_african_american',
                  'next_egfr_mdrd_african_american_max', 'next_egfr_mdrd_african_american_min',
                  'next_egfr_mdrd_non_african_american', 'next_egfr_mdrd_non_african_american_max',
                  'next_egfr_mdrd_non_african_american_min'], axis=1)
data = data.dropna()

if diagnosis_reward:
    label_cols = list(data.columns[116:159])
else:
    label_cols = list(data.columns[117:160])

full_label_cols = label_cols
if target_treatment != None:
    label_cols = treatment_ctg[target_treatment]
    action_size = len(label_cols)

data = data.loc[data[label_cols].sum(axis=1, skipna=True) != 0,]
target = data[label_cols].apply(lambda x: hash_to_action(x), axis=1)
data['target'] = target
counter = collections.Counter(target)
target_set = list([i[0] for i in counter.most_common(action_size)])
target_replacement = dict(zip(iter(target_set), range(action_size)))
target_column_renames = ['target' + str(i) for i in range(action_size)]
data = data[data.target.apply(lambda x: x in target_set)]
data['target'] = data['target'].replace(target_replacement)
reward_cols = ['reward']


/kaggle/input/cleaned-ehr-treatment-v6/cleaned_EHR_treatment_param_lab_test_final_3diseases-v6.csv


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
next_state_cols = ['next_creatinine',
                   'next_bmi',
                   'next_hemoglobin_a1c',
                   'next_antihyperlipidemic-hmg_coa_reductase_inhib.-niacin_hist',
                   'next_hemoglobin_a1c_max',
                   'next_bp_diastolic_min',
                   'next_antihyperglycemic,_sglt-2_and_dpp-4_inhibitor_comb_cur',
                   'next_antihyperglycemic,_dpp-4_inhibitors_hist',
                   'next_race_white',
                   'next_antihyperglycemc-sod/gluc_cotransport2(sglt2)inhib_hist',
                   'next_ldl_cholesterol_min',
                   'next_antihyperlip.hmg_coa_reduct_inhib-cholest.ab.inhib_cur',
                   # 'next_egfr_mdrd_non_african_american_min',
                   'next_alpha/beta-adrenergic_blocking_agents_cur',
                   'next_antihyperglycemic,insulin-release_stim.-biguanide_cur',
                   'next_antihypergly,insulin,long_act-glp-1_recept.agonist_cur',
                   'next_angioten.receptr_antag-calcium_chanl_blkr-thiazide_cur',
                   'next_anti-inflammatory,_interleukin-1_beta_blockers_cur',
                   'next_beta-blockers_and_thiazide,thiazide-like_diuretics_hist',
                   'next_antihyperglycemic,_thiazolidinedione_and_biguanide_hist',
                   'next_antihypertensives,_ace_inhibitors_cur',
                   # 'next_egfr_mdrd_african_american',
                   # 'next_egfr_mdrd_non_african_american',
                   'next_race_native_hawaiian_or_other_pacific_islander',
                   'next_antihypertensives,_angiotensin_receptor_antagonist_hist',
                   'next_antihyperglycemic,_insulin-release_stimulant_type_cur',
                   'next_antihypergly,incretin_mimetic(glp-1_recep.agonist)_cur',
                   'next_antihyperglycemic,insulin-release_stim.-biguanide_hist',
                   'next_angiotensin_receptor_blockr-calcium_channel_blockr_hist',
                   'next_antihyperglycemic,_biguanide_type_cur',
                   'next_antihyperglycemic-sglt2_inhibitor-biguanide_combs._cur',
                   'next_creatinine_min',
                   'next_bmi_min',
                   # 'next_egfr_mdrd_african_american_max',
                   'next_antihyperglycemic_-_dopamine_receptor_agonists_cur',
                   'next_beta-adrenergic_blocking_agents_hist',
                   'next_antihyperglycemic_-_dopamine_receptor_agonists_hist',
                   'next_insulins_cur',
                   'next_race_native_american',
                   'next_hdl_cholesterol',
                   'next_race_multiple_race',
                   'next_antihyperglycemic,_sglt-2_and_dpp-4_inhibitor_comb_hist',
                   'next_antihyperlipidemic_-_hmg_coa_reductase_inhibitors_cur',
                   'next_renin_inhibitor,direct-angiotensin_receptr_antagon_hist',
                   'next_anti-inflammatory,_interleukin-1_beta_blockers_hist',
                   'next_angioten.receptr_antag-calcium_chanl_blkr-thiazide_hist',
                   'next_antihyperlipidemic-hmg_coa_reductase_inhib.-niacin_cur',
                   'next_triglycerides_max',
                   'next_antihyperglycemic-sglt2_inhibitor-biguanide_combs._hist',
                   'next_hemoglobin_a1c_min',
                   'next_triglycerides',
                   # 'next_egfr_mdrd_non_african_american_max',
                   'next_miotics_and_other_intraocular_pressure_reducers_hist',
                   'next_potassium_sparing_diuretics_in_combination_cur',
                   'next_bmi_max',
                   'next_beta-adrenergic_blocking_agents_cur',
                   'next_race_other',
                   'next_time_last_vist',
                   'next_antihyperglycemic,_thiazolidinedione_and_biguanide_cur',
                   'next_bp_systolic',
                   'next_bile_salt_sequestrants_cur',
                   'next_antihyperglycemic,dpp-4_inhibitor-biguanide_combs._cur',
                   'next_angiotensin_recept-neprilysin_inhibitor_comb(arni)_cur',
                   'next_race_patient_refused',
                   'next_antihyperglycemic,_amylin_analog-type_cur',
                   'next_antihyperglycemic,_alpha-glucosidase_inhibitors_cur',
                   'next_antihypergly,dpp-4_enzyme_inhib.-thiazolidinedione_cur',
                   'next_renin_inhibitor,direct_and_thiazide_diuretic_comb_cur',
                   'next_angiotensin_receptor_antag.-thiazide_diuretic_comb_hist',
                   'next_antihyperglycemic,thiazolidinedione(pparg_agonist)_cur',
                   'next_cholesterol,_total',
                   'next_antihyperlipid-_hmg-coa_ri-calcium_channel_blocker_cur',
                   'next_thiazide_and_related_diuretics_hist',
                   'next_calcium_channel_blocking_agents_hist',
                   'next_hdl_cholesterol_max',
                   'next_bulk_chemicals_hist',
                   'next_antihyperlipidemic_-_hmg_coa_reductase_inhibitors_hist',
                   'next_potassium_sparing_diuretics_in_combination_hist',
                   'next_hdl_cholesterol_min',
                   # 'next_egfr_mdrd_african_american_min',
                   'next_race_asian',
                   'next_cholesterol,_total_max',
                   'next_antihyperlipid-_hmg-coa_ri-calcium_channel_blocker_hist',
                   'next_ace_inhibitor-thiazide_or_thiazide-like_diuretic_cur',
                   'next_ace_inhibitor-thiazide_or_thiazide-like_diuretic_hist',
                   'next_antihyperglycemic,_alpha-glucosidase_inhibitors_hist',
                   'next_antihyperglycemic,dpp-4_inhibitor-biguanide_combs._hist',
                   'next_lipotropics_hist',
                   'next_antihypergly,dpp-4_enzyme_inhib.-thiazolidinedione_hist',
                   'next_cholesterol,_total_min',
                   'next_smoke',
                   'next_ldl_cholesterol_max',
                   'next_alpha/beta-adrenergic_blocking_agents_hist',
                   'next_antihyperglycemic,_insulin-release_stimulant_type_hist',
                   'next_antihyperlipidemic_-_pcsk9_inhibitors_hist',
                   'next_antihypertensives,_angiotensin_receptor_antagonist_cur',
                   'next_renin_inhibitor,direct_and_thiazide_diuretic_comb_hist',
                   'next_bulk_chemicals_cur',
                   'next_angiotensin_receptor_antag.-thiazide_diuretic_comb_cur',
                   'next_antihyperglycemc-sod/gluc_cotransport2(sglt2)inhib_cur',
                   'next_antihyperglycemic,thiazolidinedione(pparg_agonist)_hist',
                   'next_bile_salt_sequestrants_hist',
                   # 'next_reward',
                   'next_creatinine_max',
                   'next_renin_inhibitor,direct-angiotensin_receptr_antagon_cur',
                   'next_angiotensin_receptor_blockr-calcium_channel_blockr_cur',
                   'next_antihyperglycemic,_thiazolidinedione-sulfonylurea_cur',
                   'next_antihyperlip.hmg_coa_reduct_inhib-cholest.ab.inhib_hist',
                   'next_antihyperglycemic,_biguanide_type_hist',
                   'next_sex_male',
                   'next_antihyperglycemic,_amylin_analog-type_hist',
                   'next_race_unknown',
                   'next_antihypertensives,_ace_inhibitors_hist',
                   'next_antihyperglycemic,_thiazolidinedione-sulfonylurea_hist',
                   'next_antihyperglycemic,_dpp-4_inhibitors_cur',
                   'next_beta-blockers_and_thiazide,thiazide-like_diuretics_cur',
                   'next_miotics_and_other_intraocular_pressure_reducers_cur',
                   'next_triglycerides_min',
                   'next_bp_diastolic',
                   'next_bp_systolic_max',
                   'next_bp_systolic_min',
                   'next_age',
                   'next_calcium_channel_blocking_agents_cur',
                   'next_antihyperlipidemic_-_pcsk9_inhibitors_cur',
                   'next_ldl_cholesterol',
                   'next_antihypergly,insulin,long_act-glp-1_recept.agonist_hist',
                   'next_insulins_hist',
                   'next_thiazide_and_related_diuretics_cur',
                   'next_angiotensin_recept-neprilysin_inhibitor_comb(arni)_hist',
                   'next_lipotropics_cur',
                   'next_antihypergly,incretin_mimetic(glp-1_recep.agonist)_hist',
                   'next_bp_diastolic_max',
                   'next_time_to_first_visit']
state_cols = [s[5:] for s in next_state_cols]
# state_cols = list(set(data.columns) - set(full_label_cols) - {'target'} - set(
#     ['reward', 'reward_bp', 'reward_ascvd', 'reward_diabetes', 'risk_ascvd', 'next_risk_ascvd']) - set(
#     ['study_id', 'encounter_dt_ran']) - set(next_state_cols))
if diagnosis_reward:
    state_cols = state_cols - (['CVD', 'days_to_CVD'])

# next_state_cols = ['next_' + s for s in list(state_cols)]
patients_column = data[['study_id', 'encounter_dt_ran']]
#data = data.drop(excluded, axis=1)
_temp = data.drop('target', axis=1).max(skipna=True) - data.drop('target', axis=1).min(skipna=True)
_temp[data.drop('target', axis=1).columns[(_temp == 0).values]] = 1.0
normalized_df = (data.drop(['target'], axis=1) - data.drop('target', axis=1).min(skipna=True)) / _temp

# constant_variable_df = data.drop('target', axis=1).max(skipna=True) - data.drop('target', axis=1).min(skipna=True)
# dropped_state = normalized_df.columns[(constant_variable_df == 0).values]
# normalized_df[dropped_state] = data[dropped_state]
# normalized_df['reward'] = (data['reward'] - data['reward'].min(skipna=True))/(data['reward'].std()*2)
# next_state_cols = set(next_state_cols) - set(dropped_state)
# state_cols = set(state_cols) - set(dropped_state)
ohe = to_categorical(data['target'], action_size)

for i, col in enumerate(target_column_renames):
    normalized_df[col] = ohe[:, i]
normalized_df[['study_id', 'encounter_dt_ran']] = patients_column
train, test = train_test_split(normalized_df, test_size=0.4, random_state=2019)
train2, test2 = train_test_split(data, test_size=0.4, random_state=2019)

state_size = len(state_cols)
action_size = len(target_column_renames)
# loss_weights = loss_weight(data)
#class_weights = class_weight.compute_class_weight('balanced',
#                                                  np.unique(data['target'].values),
#                                                  data['target'].values)
agent = DQNAgent(state_size, action_size, target_column_renames, state_cols, reward_cols, next_state_cols)#, class_weights, True)
#agent.load("./model/3d:256-512-256-episodes:20000--dqn-mse-target_treatment:{}.h5".format(target_treatment))
done = False
batch_size = 64
cur_val_error = 0
episodes_till_target_update = 100
interested_train = train  # train[train[targets].apply(lambda x: sum(x) > 0, axis=1)]
interested_tests = test  # test[test[targets].apply(lambda x: sum(x) > 0, axis=1)]
sample_patient = False

In [3]:
for e in range(EPISODES):

    # patients_trajectories = random.sample(patients_set, batch_size)
    if sample_patient:
        patient = random.sample(patients_set, 1)
        minibatch = interested_train[interested_train['study_id'] == patient]
    else:
        minibatch = interested_train.sample(batch_size)

    # minibatch = random.sample(agent.memory, batch_size)
    if e % episodes_till_target_update:
        agent.update_target()

    # for visit in minibatch:
    #     loss = agent.replay(minibatch[visit], True)

    agent.learning_rate = agent.learning_rate * (1 - agent.learning_rate_decay)
    loss = agent.replay(minibatch, True)

    if e % 500 == 0:
        cur_val_error = sum(np.argmax(interested_tests[target_column_renames].values, axis=1) == np.argmax(
            agent.model.predict(interested_tests[state_cols]), axis=1)) / interested_tests.shape[0]
    # Logging training loss every 10 timesteps
    if e % 10 == 0:
        print("episode: {}/{}, loss: {:.7f}, test_acc: {:.4f}"
              .format(e, EPISODES, loss, cur_val_error))
            
agent.save("/kaggle/input/episodes20000dqnmsetarget-hypertension/3d_256-512-256-episodes_20000--dqn-mse-target_treatment_{}.h5".format(target_treatment))

episode: 0/20000, loss: 2.7728093, test_acc: 0.0553
episode: 10/20000, loss: 2.7548699, test_acc: 0.0553
episode: 20/20000, loss: 1.7978199, test_acc: 0.0553
episode: 30/20000, loss: 3.0415230, test_acc: 0.0553
episode: 40/20000, loss: 3.0688909, test_acc: 0.0553
episode: 50/20000, loss: 2.1194037, test_acc: 0.0553
episode: 60/20000, loss: 1.2825356, test_acc: 0.0553
episode: 70/20000, loss: 1.1207505, test_acc: 0.0553
episode: 80/20000, loss: 1.9501070, test_acc: 0.0553
episode: 90/20000, loss: 1.7698300, test_acc: 0.0553
episode: 100/20000, loss: 1.4021145, test_acc: 0.0553
episode: 110/20000, loss: 1.1607285, test_acc: 0.0553
episode: 120/20000, loss: 0.8137211, test_acc: 0.0553
episode: 130/20000, loss: 1.1347305, test_acc: 0.0553
episode: 140/20000, loss: 0.7962626, test_acc: 0.0553
episode: 150/20000, loss: 1.0082760, test_acc: 0.0553
episode: 160/20000, loss: 0.7728631, test_acc: 0.0553
episode: 170/20000, loss: 1.4407499, test_acc: 0.0553
episode: 180/20000, loss: 0.8584723, te

OSError: Unable to create file (unable to open file: name = '/kaggle/input/episodes20000dqnmsetarget-hypertension/3d_256-512-256-episodes_20000--dqn-mse-target_treatment_ascvd.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)